# Работа с ошибками в словах.
При распознавании текстов из формата jpeg сопровождается большим количеством ошибок в словах.
Это связанно с тем что иногда изображения очень грязные и текст выделяется не очень четко.
Я пытаюсь создать методы которые позволяют максимально автоматизировать процесс исправления ошибок.
В этом варианте я рассматриваю словарь в котором созданный на основе распознанного текста с частотой встречаемости слов.

In [1]:
# sources words 
name_source = ['/home/al/Projects_My/NLP-russian-language/NLP_gansior/FIO/datasets/subnames_base_com.csv',
              '/home/al/Projects_My/recognition_text/file_name_org_11_01_2022.txt',
              '/home/al/Projects_My/recognition_text/0 ВСЕ СКАНЫfile_text_model2.txt']
name_result = ['/home/al/Projects_My/NLP-russian-language/NLP_gansior/file_corrected_errors_16_01_2022.csv',
              '/home/al/Projects_My/recognition_text/0 ВСЕ СКАНЫfile_text_model2_correct_18_01_2022.txt']
import string
import re
import pymorphy2
from collections import Counter

morph = pymorphy2.MorphAnalyzer()

Loading dictionaries from /usr/local/lib/python3.8/dist-packages/pymorphy2_dicts_ru/data
format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [2]:
def preprocess_text(sen):

    # Remove punctuations and numbers
    sentence = re.sub('[^а-яё]', ' ', sen.lower())

    # Single character removal
    #sentence = re.sub(r"\s+[а-яёА-ЯЁ]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [3]:
def get_words_vocab_file(n_f:str)-> list:
    answer = []

    with open(n_f, 'r') as i_f:
        for line in i_f:
            s_l = line.strip().lower().split('^')
            answer.append(s_l)
    return answer

In [4]:
# подготовка данных из специфического набора.
def get_txt_vdnh(nam_f:str)-> list:
    i_f = open(nam_f, 'r')
    full_txt = i_f.read()
    l_t = full_txt.split('coord_block ')
    l_t = [preprocess_text(s) for s in l_t]
    return l_t

In [5]:
def get_words_vocab_rem(n_f:list)-> list:
    answer = []
    voc = Counter()
    for line in n_f:
        line = re.sub(r'\s+', ' ', line)
        s_l = line.strip().split(' ')
        for ww in s_l:
            voc[ww.strip()] +=1
    for word in voc:
        if len(word)>= 3:
            answer.append([word, voc[word]])
    return answer

In [6]:
def get_words_frag_beg(i_l:list, frag:str) ->list:
    answer =[]
    br = len(frag)
    for kk in i_l:
        if kk[0][:br] in [frag]:
            answer.append(kk)
    return answer

In [7]:
full_list_txt_vdnh = get_txt_vdnh(name_source[2])

In [82]:
data_vdnh = get_words_vocab_rem(full_list_txt_vdnh)
print(len(data_vdnh))

244620


In [83]:
import operator

In [84]:
def count_char(word:str):
    answer = True
    cc = Counter(word).most_common(3)
    if cc[0][1]>4 : answer = False
    return answer    

In [85]:
print(len(data_vdnh))
print(data_vdnh[:10])
data_vdnh = [x for x in data_vdnh if len(x[0]) >= 4 and count_char(x[0])]
data_vdnh.sort(key=lambda x:len(x[0]), reverse=True)
#data_vdnh = sorted(data_vdnh, key=operator.itemgetter(1), reverse=True)
print('##################################')
print(data_vdnh[:10])
print(len(data_vdnh))

244620
[['ноябрь', 2335], ['все', 19473], ['сканы', 19085], ['организацию', 1828], ['торговли', 188], ['товарами', 5], ['сложжно', 1], ['ассортимента', 53], ['заказам', 13], ['населения', 231]]
##################################
[['бензолсульфамидмочевиноформальдегидных', 2], ['мочевинобензолсульфамидформальдегидных', 2], ['слектромонтажконструкцияминспецстроя', 1], ['ремстройдорыашёсёёавулъяновскстроя', 1], ['экспериментальновконструкторскому', 1], ['экспериментальноисследовательских', 1], ['алексеевичатрубоэлектросварцщика', 1], ['экспериментальноконструкторский', 1], ['экспериментальноконструкторское', 1], ['квалификационнопрофессиональная', 1]]
233358


In [58]:
def del_entry_words(data_vdnh):
    k = len(data_vdnh)
    while k > 5:
        word = data_vdnh[k-1][0]
        del_key = False
        #print(word)
        for ww in data_vdnh:
            if ww[0].find(word)>=0:
                if data_vdnh.index(ww) != (k-1):
                    #print(data_vdnh.index(ww))
                    #print(ww[0], word)
                    del_key = True
                    break
        if del_key :
            del data_vdnh[k-1]
        k -=1
        if k%1000 == 0: 
            print(k, len(data_vdnh))
            #break
    return data_vdnh

In [59]:
data_vdnh = del_entry_words(data_vdnh)
print(len(data_vdnh))

244000 244516
243000 244307
242000 244092
241000 243926
240000 243714
239000 243529
238000 243339
237000 243173
236000 242968
235000 242731
234000 242527
233000 242339
232000 242108
231000 241906
230000 241672
229000 241425
228000 241072
227000 240874
226000 240685
225000 240487
224000 240255
223000 240014
222000 239771
221000 239497
220000 239249
219000 239083
218000 238889
217000 238655
216000 238376
215000 238143
214000 237941
213000 237714
212000 237464
211000 237273
210000 237077
209000 236876
208000 236694
207000 236490
206000 236300
205000 236076
204000 235855
203000 235666
202000 235468
201000 235274
200000 235035
199000 234817
198000 234621
197000 234433
196000 234229
195000 234061
194000 233872
193000 233663
192000 233478
191000 233283
190000 233136
189000 232930
188000 232700
187000 232507
186000 232293
185000 232102
184000 231905
183000 231722
182000 231554
181000 231437
180000 231191
179000 231011
178000 230817
177000 230570
176000 230362
175000 230119
174000 229938
173000

In [60]:
print(data_vdnh[:50])

[['рублей', 32981], ['размере', 25503], ['получение', 25005], ['процентов', 24757], ['премией', 24380], ['району', 2643], ['ноябрь', 2335], ['первой', 1516], ['непосредственное', 1273], ['успешное', 949], ['опытнической', 863], ['методики', 779], ['пятъдесят', 718], ['искусственному', 695], ['наградить', 683], ['ячменя', 669], ['квадратного', 662], ['почета', 612], ['свиноматку', 608], ['квадратных', 601], ['ольгу', 518], ['калинина', 506], ['каждые', 492], ['комплексного', 480], ['доводку', 466], ['каракульских', 462], ['советского', 439], ['повышению', 430], ['машину', 424], ['отработку', 421], ['теплицах', 409], ['специального', 408], ['антонину', 404], ['которого', 400], ['виталия', 390], ['приспособлений', 382], ['параметров', 381], ['творчества', 363], ['улучшить', 361], ['передовых', 360], ['искусственного', 352], ['чертежей', 348], ['отдельных', 345], ['республики', 340], ['раису', 338], ['рационализаторских', 337], ['ручной', 328], ['денекной', 324], ['показателей', 316], ['ус

In [68]:
f_r = open('/home/al/Projects_My/recognition_text/vocab_first_step_clearing_25_01_2022.csv', 'w')
for ww in data_vdnh:
    if len(ww[0])<=50:
        f_r.write(ww[0] + '^' + str(ww[1]) + '\n')
f_r.close()

In [67]:
data_vdnh.sort(key=lambda x:len(x[0]), reverse=True)
print(data_vdnh[:50])

[['тосроачотаичаоноооаобноеоренеонечооанонссноточеночноононыоанодонааодонинонночооанарнящоереочнарченнччаеуноаееноноиоаеосодаоенрионннднощиоаеннуини', 1], ['иномалоощрщинщициюанолееотороллеомолсимеслоаяаищляншищеувищиетоомоциишемртишепицолаиоомещотоеооритощиаощотолщиооооцчеацмолоииелаортмоооощино', 1], ['ститеносиоощонеооонооеннолнитоелоооцитаоттеттининеоеоостоовеетоооонецеоанеоатоеоолоончонооеноеочетоно', 1], ['етаноесототиашанноиоенщниеионявоанооноетсоененоечиеронтттянеищиевнооаосооннооемнтееееорото', 1], ['венинениииоминисватнмоотелиениитииещещенилочетелтелсииетоилеетьетонеьетететлтисвтциинвттн', 1], ['итемыщеиотесотыемостетаотееетесеитчмиеииептичтиеещттетриниеевииоиетослтемисестииеметоои', 1], ['ннннееенньненеоннонненностеесиотонениионатчовнноьнностонненкоеоаытомоьнонототоаноооноя', 1], ['еттешитееонеелиоииеиншснонщооаиетениеоиноехиненяиееоееиотоотаонетотоаонеощоетеволонето', 1], ['сареоощаненотеоостнннинтетттоаеоеотчфенточоееотиутототьеоиесттатеноннесчснейсснайнст', 1], ['тсоеноль

In [10]:
# получаем самое частое слово длиной больше равно 4
dd =[0,0,'']
#print(l_frag[10])
for kk in range(len(data_vdnh)):
    if data_vdnh[kk][1]>dd[1] and len(data_vdnh[kk][0])>=4 :
        dd = [kk,data_vdnh[kk][1], data_vdnh[kk][0]]
print(dd)

[199, 32981, 'рублей']


In [11]:
def vocab_word(word:str)-> dict:
    vv={}
    rual = ''.join([chr(ord('а')+ kk) for kk in range(32)])+'ё'
    for k in range(2,len(word)-1):
        if word[:k]:
            vv[word[:k]] = word
        if word[k:]:
            vv[word[k:]] = word
    return vv

In [12]:
def vocab_word_break(word:str)-> dict:
    vv={word:[]}
    #rual = ''.join([chr(ord('а')+ kk) for kk in range(32)])+'ё'
    for k in range(1,len(word)):
        if word[:k] and word[k:]:
            vv[word].append(word[:k] + ' ' + word[k:])
    return vv

In [13]:
err_word_dict = vocab_word(dd[2])
print(len(err_word_dict))
print(err_word_dict)

6
{'ру': 'рублей', 'блей': 'рублей', 'руб': 'рублей', 'лей': 'рублей', 'рубл': 'рублей', 'ей': 'рублей'}


In [14]:
for kk in range(len(data_vdnh)):
    if data_vdnh[kk][0] in err_word_dict:
        print(data_vdnh[kk][0])

руб
лей
ру
блей
рубл
ей


In [15]:
word_break = vocab_word_break(dd[2])
print(len(word_break[dd[2]]))
print(word_break[dd[2]])

5
['р ублей', 'ру блей', 'руб лей', 'рубл ей', 'рубле й']


In [16]:
def chan_text(is_txt:list, word:str ,br_ls:list) ->list:
    answer = []
    for sen in is_txt:
        for br_w in br_ls:
            sen = sen.replace(br_w,' ' + word + ' ')
            sen = re.sub(r'\s+', ' ', sen)
        answer.append(sen)
    return answer

In [17]:
senten = chan_text(full_list_txt_vdnh, dd[2] , word_break[dd[2]])

In [18]:
data_vdnh = get_words_vocab_rem(senten)
print(len(data_vdnh))
print(data_vdnh[:10])

245716
[['ноябрь', 2335], ['все', 19473], ['сканы', 19085], ['п', 23458], ['н', 22362], ['с', 38058], ['за', 88219], ['организацию', 1828], ['торговли', 188], ['товарами', 5]]


In [19]:
# получаем самое частое слово длиной больше равно 4
dd =[0,0,'']
#print(l_frag[10])
for kk in range(len(data_vdnh)):
    if data_vdnh[kk][1]>dd[1] and len(data_vdnh[kk][0])>=4 :
        dd = [kk,data_vdnh[kk][1], data_vdnh[kk][0]]
print(dd)

[199, 33090, 'рублей']


In [20]:
for kk in range(len(data_vdnh)):
    if data_vdnh[kk][0] in err_word_dict:
        print(data_vdnh[kk][0])

руб
лей
ру
блей
рубл
ей


In [21]:
def clear_txt(is_txt:list)-> list:
    data_vdnh = get_words_vocab_rem(is_txt)
    # очистка от разрывов слов
    word_clear = []
    work_key =True
    k = 0
    while work_key :
        dd =[0,0,'']
        #print(l_frag[10])
        for kk in range(len(data_vdnh)):
            if data_vdnh[kk][1]>dd[1] and len(data_vdnh[kk][0])>=4 and data_vdnh[kk][0] not in word_clear:
                dd = [kk,data_vdnh[kk][1], data_vdnh[kk][0]]
        if dd[2]:
            word_clear.append(dd[2])
            print(dd)
            word_break = vocab_word_break(dd[2])
            is_txt = chan_text(is_txt, dd[2], word_break)
            data_vdnh = get_words_vocab_rem(is_txt)
            print('data_vdnh = ', len(data_vdnh))
            k +=1
            if k == 100 : work_key = False
        else:
            work_key = False
    return is_txt

In [22]:
full_list_txt_vdnh = clear_txt(full_list_txt_vdnh)

[199, 32981, 'рублей']
data_vdnh =  245694
[197, 25503, 'размере']
data_vdnh =  245687
[2861, 25005, 'получение']
data_vdnh =  245667
[2907, 24757, 'процентов']
data_vdnh =  245639
[196, 24380, 'премией']
data_vdnh =  245629
[194, 24367, 'медалью']
data_vdnh =  245623
[195, 24233, 'денежной']
data_vdnh =  245607
[511, 23014, 'разработку']
data_vdnh =  245593
[2901, 21572, 'центнеров']
data_vdnh =  245560
[2890, 20910, 'сбора']
data_vdnh =  245544
[2981, 20910, 'процента']
data_vdnh =  245513
[39, 20850, 'внедрение']
data_vdnh =  245484
[2889, 20045, 'валового']
data_vdnh =  245468
[2, 19085, 'сканы']
data_vdnh =  245468
[2896, 16956, 'урожая']
data_vdnh =  245458
[2873, 16824, 'гектара']
data_vdnh =  245440
[2907, 14671, 'молока']
data_vdnh =  245416
[3073, 14444, 'центнера']
data_vdnh =  245394
[282, 13358, 'бронзовой']
data_vdnh =  245391
[1015, 12672, 'колхоза']
data_vdnh =  245365
[51, 10807, 'плана']
data_vdnh =  245326
[2901, 10715, 'зерна']
data_vdnh =  245299
[2903, 10210, 'мяс

In [23]:
f_r_sent = open(name_result[1], 'w')
for ssent in full_list_txt_vdnh:
    f_r_sent.write(ssent + '\n')

In [ ]:
beg_data = get_words_vocab_file(name_source[0])

In [23]:
#l_frag = get_words_frag_beg(beg_data, 'уч')
dd =[0,0,'']
#print(l_frag[10])
for kk in range(len(beg_data)):
    if int(beg_data[kk][1])>dd[1]:
        dd = [kk,int(beg_data[kk][1]), beg_data[kk][0]]
print(dd)

[5053, 4427, 'ивановича']


In [24]:
err_word_dict = vocab_word(dd[2])
print(len(err_word_dict))
print(err_word_dict)

12
{'ив': 'ивановича', 'ановича': 'ивановича', 'ива': 'ивановича', 'новича': 'ивановича', 'иван': 'ивановича', 'овича': 'ивановича', 'ивано': 'ивановича', 'вича': 'ивановича', 'иванов': 'ивановича', 'ича': 'ивановича', 'иванови': 'ивановича', 'ча': 'ивановича'}


In [25]:
word_break = vocab_word_break(dd[2])
print(len(word_break[dd[2]]))
print(word_break[dd[2]])

6
['ив ановича', 'ива новича', 'иван овича', 'ивано вича', 'иванов ича', 'иванови ча']


In [26]:
for kk in range(len(beg_data)):
    if beg_data[kk][0] in err_word_dict:
        print(beg_data[kk][0])

ановича
вича
ива
иван
ивано
иванов
иванови
ича
новича
овича
ча
